- Họ tên: Võ Minh Tung
- Mã số học viên: 2370345

1. Install Dependencies

In [1]:

%pip install transformers datasets torch torchvision evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


 2. Prepare the MNIST Dataset

In [ ]:
from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras import layers, models

# Tải bộ dữ liệu MNIST từ Hugging Face
dataset = load_dataset("mnist")

# Chuyển đổi dữ liệu thành định dạng TensorFlow
def preprocess_function(example):
    image = tf.keras.preprocessing.image.img_to_array(example['image'])  # PIL -> NumPy array
    # Chuyển đổi hình ảnh từ đối tượng PIL thành TensorFlow tensor
    image = tf.convert_to_tensor(image)  # Đảm bảo là tensor
    image = tf.image.convert_image_dtype(image, tf.float32)  # Chuẩn hóa giá trị [0, 1]
    image = tf.image.resize(image, (28, 28))  # Đảm bảo kích thước là 28x28
    if image.shape[-1] != 1:  # Thêm kênh màu nếu chưa có (28, 28) -> (28, 28, 1)
        image = tf.expand_dims(image, axis=-1)
    label = tf.cast(example['label'], tf.int32)  # Đảm bảo nhãn là kiểu int
    return {"image": image, "label": label}  # Trả về dạng dictionary với các cột

# Áp dụng preprocessing cho tập huấn luyện và kiểm tra
train_dataset = dataset['train'].map(preprocess_function, remove_columns=["image", "label"]).with_format("tensorflow")
test_dataset = dataset['test'].map(preprocess_function, remove_columns=["image", "label"]).with_format("tensorflow")

#print(preprocess_function(dataset['train'][0]))

Map:   2%|▏         | 999/60000 [00:27<26:42, 36.82 examples/s]  


KeyboardInterrupt: 

3. Load a Pretrained Vision Transformer

In [10]:
from transformers import ViTForImageClassification, ViTFeatureExtractor

# Load feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=10,  # MNIST has 10 classes
    id2label={i: str(i) for i in range(10)},  # Map IDs to digit labels
    label2id={str(i): i for i in range(10)}
)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4. Prepare the DataLoader

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)


5. Fine-tune the Model

In [ ]:
import torch
from torch.optim import AdamW
from transformers import get_scheduler

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * 3  # Assuming 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Define loss function and move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
model.train()
for epoch in range(3):  # Train for 3 epochs
    for batch in train_dataloader:
        # Kiểm tra dữ liệu
        #print(type(batch["image"]))  # Kiểm tra kiểu dữ liệu

        # Nếu batch["image"] là list
        if isinstance(batch["image"], list):
            inputs = torch.stack(batch["image"]).to(device)
        else:
        # Nếu batch["image"] đã là tensor
            inputs = batch["image"].to(device)

        labels = torch.tensor(batch["label"]).to(device)
        outputs = model(pixel_values=inputs, labels=labels)
        loss = outputs.loss
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()


    print(f"Epoch {epoch + 1} complete. Loss: {loss.item():.4f}")


<class 'torch.Tensor'>


C:\Users\oum1hc\AppData\Local\Temp\ipykernel_4192\291651108.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"]).to(device)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
Epoch 1 complete. Loss: 1.8444
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
Epoch 2 complete. Loss: 1.4378
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tens

6. Evaluate the Model

In [17]:
import evaluate

# Load metric
metric = evaluate.load("accuracy")

# Evaluation loop
model.eval()
for batch in test_dataloader:
    
    if isinstance(batch["image"], list):
        inputs = torch.stack(batch["image"]).to(device)
    else:
        # Nếu batch["image"] đã là tensor
        inputs = batch["image"].to(device)

    labels = torch.tensor(batch["label"]).to(device)
    
    with torch.no_grad():
        outputs = model(pixel_values=inputs)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

# Compute final accuracy
final_accuracy = metric.compute()["accuracy"]
print(f"Test Accuracy: {final_accuracy * 100:.2f}%")



C:\Users\oum1hc\AppData\Local\Temp\ipykernel_4192\3368052241.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"]).to(device)


Test Accuracy: 80.00%
